In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import metrics
import random

In [2]:
df = pd.read_csv("ConsultIT_HelpDesk_Tickets.csv")

In [3]:
feature_cols = ['RequestorSeniority', 'ITOwner', 'FiledAgainst', 'TicketType', 'Severity', 'Priority']
X = df[feature_cols]

feature_target = ['response']
y = df[feature_target]

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=24, test_size=0.2)

In [5]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [6]:
%%capture
dt = DecisionTreeClassifier()
dt.fit(X_train,y_train)

In [7]:
import warnings
warnings.filterwarnings('ignore')

import ipywidgets as widgets
from IPython.display import display, clear_output

In [8]:
y_pred = dt.predict(X_test)

In [9]:
Ticket_catagories = [
    'Access/Login', 'Hardware', 'Software', 
    'Systems'
]

In [10]:
#print('Train Accuracy: ', accuracy_score(y_train,dt.predict(X_train)))

In [11]:
#!jupyter nbextension enable --py widgetsnbextension --sys-prefix
#!jupyter serverextension enable voila --sys-prefix

In [12]:
RequestorSeniority= widgets.Dropdown(
    options=[('Junior', 1), ('Regular', 2), ('Senior', 3), ('Management', 4)],
    value=2,
)

In [13]:
FiledAgainst= widgets.Dropdown(
    options=[('Access/Login', 0), ('Hardware', 1), ('Software', 2), ('Systems', 3)],
    value=0,
)

In [14]:
TicketType= widgets.Dropdown(
    options=[('Issue', 0), ('Request', 1)],
    value=0,
)

In [15]:
Severity= widgets.Dropdown(
    options=[('Unsure', 0), ('Minor', 1), ('Normal', 2), ('Major', 3), ('Critical', 4)],
    value=0,
)

In [16]:
Priority= widgets.Dropdown(
    options=[('Unassigned', 0), ('Low', 1), ('Medium', 2), ('High', 3)],
    value=0,
)

In [17]:
text_0 = widgets.HTML(value="<h1>ConsultIT</h1>")
text_1 = widgets.HTML(value="<h3>Thank you for trusting ConsultIT with your IT assistance needs</h3>")
text_2 = widgets.HTML(value="<h3>Please fill out the information below and hit Submit Ticket when complete</h3>")
text_3 = widgets.HTML(value="<h4>Which area is your ticket related to:</h4>")
text_4 = widgets.HTML(value="<h4>What type of ticket is this:</h4>")
text_5 = widgets.HTML(value="<h4>What would you say is the severity of this ticket:</h4>")
text_6 = widgets.HTML(value="<h4>How high of a priorty would you relate your ticket:</h4>")
text_7 = widgets.HTML(value="<h4>What is your position level:</h4>")

In [18]:
estimated_completion = ''
def estimated_time(prediction):
    if prediction == 1:
        estimated_completion = '0 - 4'
    elif prediction == 2:
        estimated_completion = '5 - 10'
    elif prediction == 3:
        estimated_completion = '11 - 25'
    else:
        estimated_completion = '26+'
    return estimated_completion

In [19]:
def pie_chart():
    num_catagories = df.groupby(by='FiledAgainst').size()
    plt.figure(figsize=(8,8))
    plt.pie(x=num_catagories, labels=Ticket_catagories,autopct='%1.2f%%',textprops={'fontsize':14})
    plt.title(
        label="Percentage of Trouble Tickets filed by Catagory", 
        fontdict={"fontsize":16},
        pad=20
    )
    plt.show()

In [20]:
button_send = widgets.Button(
                description='Submit Ticket',
                tooltip='Send',
                style={'description_width': 'initial'}
            )

output = widgets.Output()

def on_button_clicked(event):
    with output:
        clear_output()
        ITOwner = random.randint(1,50)
        input_values = [RequestorSeniority.value, ITOwner, FiledAgainst.value, TicketType.value, Severity.value, Priority.value]
        response_time = dt.predict([input_values])
        date_range = estimated_time(response_time)
        print("\n")
        print(f"Thank you for submitting your ticket! This ticket has been assigned to IT assistant # {ITOwner}")
        print(f"Your issues should be resolved in {date_range} business days")
        print("\n"*3)
        pie_chart()

In [21]:
file = open("support_logo.jpg", "rb")
image = file.read()

image_headline = widgets.Image(
                    value=image,
                    format='jpg',
                    width='300'
                )

label_headline = widgets.Label(
                    value='CC0 Public Domain',
                    style={'description_width': 'initial'}
                )

vbox_headline = widgets.VBox([image_headline, label_headline])

In [22]:
button_send.on_click(on_button_clicked)

vbox_result = widgets.VBox([button_send, output])

In [23]:
vbox_text = widgets.VBox([text_0, text_1, text_2, text_3, FiledAgainst, text_4, TicketType, text_5, Severity, text_6, Priority, text_7, RequestorSeniority, vbox_result])

In [24]:
page = widgets.HBox([vbox_headline, vbox_text])

In [25]:
display(page)

In [27]:
%%capture
cm = metrics.confusion_matrix(y_test, y_pred, labels=dt.classes_)
disp = metrics.ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Rapid','Normal','Extended','Long-term'])
disp.plot()
disp.ax_.set(
                title='Response Prediciton Confusion Matrix', 
                xlabel='Predicted Response', 
                ylabel='Actual Response')

In [28]:
%%capture
plt.figure(figsize=(15,8))
sns.heatmap(df.corr(), annot=True)